In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
from tkinter import simpledialog
from tkinter import filedialog
from geopy.geocoders import Nominatim, ArcGIS,MapBox
import geopandas as gpd
from functools import  partial

%matplotlib inline

In [2]:
dirDeArchivo = filedialog.askopenfilename(title="Abrir Archivo excel")
CodRuta = simpledialog.askstring(title="Ruta o Codigo de Representante.",prompt="ingresa el codigo de reporesentante dado por Smart Scale",initialvalue="P1200").upper()
print(CodRuta)

verExcelOriginal= pd.read_excel(dirDeArchivo,0,header=1)
verExcelOriginal.head()

MEDICO


,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Categoria,Pacs por Sem,Cedula,Tel1,Celular,Email,Rango Edad,Da Consulta en Hosp,Fecha_Plan,Hora_Plan
0,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{331E754A-DF8F-4268-B7A1-725823066B40},{04C32602-3669-4BC0-A11B-9775F0FF8645},CONSULTORIOS,MEDICO,AGUILAR,GARCIA,PAMELA,LUIS LARA PARDO,...,C,DE 26 A 50,7453912,5523432964,5.523433e+09,NaN,NaN,NO,2024-07-16,00:00:00
1,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8A30491F-A3B4-4618-94F3-196F0071BC11},{62247D5F-ABB7-4F0A-8F79-18B6C7ED135A},HOSPITALES,MEDICO,AGUILAR,ZAFRA,RODOLFO,RETORNO 33 1 DE AV. DEL TALLER,...,A,MAS DE 75,607953,57688724,NaN,jrazafra@yahoo.com,66.0,NO,2024-07-12,11:00:00
2,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8E3548FB-1392-48D1-92D9-55508F0D4F4D},{B1614C2D-6B5C-4EA5-993B-7981BFF65C71},CONSULTORIOS,MEDICO,AGUIRRE,EVERARDO,LAURA,SUR 12 422,...,B,MAS DE 75,873865,5540358297,5.540358e+09,aguirreverardolaura@gmail.com,63.0,NO,2024-07-04,00:00:00
3,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{62E05F91-6FBE-4217-9CB4-BF0284FFF5B0},{66F41233-AE11-4F6A-9EF1-A4960B903F4D},CONSULTORIOS,MEDICO,AGUIRRE,HERNANDEZ,JOSE RAUL,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,...,B,DE 51 A 75,1365598,5557854347,NaN,dr.raulaguirrehdz@gmail.com,66.0,NO,2024-07-15,14:00:00
4,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{2D02B950-180B-4C61-8B0C-E4B8010CE36E},{04194637-7EF3-44B0-BCEE-6146126B8DF2},CONSULTORIOS,MEDICO,ALDAPE,MARTINEZ,GABRIELA,J. MARTINEZ ALBA 71,...,C,DE 26 A 50,2444548,5535706728,NaN,dalmarmx@yahoo.com.mx,NaN,NO,2024-07-17,09:00:00


In [3]:
verExcelOriginal.columns

verExcel = verExcelOriginal.rename(columns={'Num Ext': 'Num_Ext'})

In [4]:
verExcel['Direccion']= verExcel["Calle"]
verExcel['Col']=verExcel["Colonia"].str[7:]
verExcel['Ciudad']=verExcel["Poblacion"]
verExcel["Pais"]="Mexico"

verExcel.head()

,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Celular,Email,Rango Edad,Da Consulta en Hosp,Fecha_Plan,Hora_Plan,Direccion,Col,Ciudad,Pais
0,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{331E754A-DF8F-4268-B7A1-725823066B40},{04C32602-3669-4BC0-A11B-9775F0FF8645},CONSULTORIOS,MEDICO,AGUILAR,GARCIA,PAMELA,LUIS LARA PARDO,...,5.523433e+09,NaN,NaN,NO,2024-07-16,00:00:00,LUIS LARA PARDO,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico
1,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8A30491F-A3B4-4618-94F3-196F0071BC11},{62247D5F-ABB7-4F0A-8F79-18B6C7ED135A},HOSPITALES,MEDICO,AGUILAR,ZAFRA,RODOLFO,RETORNO 33 1 DE AV. DEL TALLER,...,NaN,jrazafra@yahoo.com,66.0,NO,2024-07-12,11:00:00,RETORNO 33 1 DE AV. DEL TALLER,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico
2,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8E3548FB-1392-48D1-92D9-55508F0D4F4D},{B1614C2D-6B5C-4EA5-993B-7981BFF65C71},CONSULTORIOS,MEDICO,AGUIRRE,EVERARDO,LAURA,SUR 12 422,...,5.540358e+09,aguirreverardolaura@gmail.com,63.0,NO,2024-07-04,00:00:00,SUR 12 422,AGRICOLA ORIENTAL,IZTACALCO,Mexico
3,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{62E05F91-6FBE-4217-9CB4-BF0284FFF5B0},{66F41233-AE11-4F6A-9EF1-A4960B903F4D},CONSULTORIOS,MEDICO,AGUIRRE,HERNANDEZ,JOSE RAUL,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,...,NaN,dr.raulaguirrehdz@gmail.com,66.0,NO,2024-07-15,14:00:00,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,GENERAL IGNACIO ZARAGOZA,VENUSTIANO CARRANZA,Mexico
4,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{2D02B950-180B-4C61-8B0C-E4B8010CE36E},{04194637-7EF3-44B0-BCEE-6146126B8DF2},CONSULTORIOS,MEDICO,ALDAPE,MARTINEZ,GABRIELA,J. MARTINEZ ALBA 71,...,NaN,dalmarmx@yahoo.com.mx,NaN,NO,2024-07-17,09:00:00,J. MARTINEZ ALBA 71,MOCTEZUMA 1 SECCION,VENUSTIANO CARRANZA,Mexico


In [5]:
def concatena_campos(row):
    return f"{row['Direccion']},{row['Col']},{row['Ciudad']},{row['Estado']},{row['Pais']}"

In [6]:
verExcel["noGeo"]= verExcel.apply(concatena_campos,axis=1)
verExcel.head()

verExcel.to_excel("archivodePrueba.xlsx")

In [7]:
verExcel_for_osm = verExcel.copy()
verExcel_for_mapbox = verExcel.copy()
verExcel_for_arcgis = verExcel.copy()

geocodificacion completa con la API de OSM

In [20]:
geolocator =Nominatim(user_agent="jorgemc",timeout=5)
geocode = partial(geolocator.geocode,language = "es")

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    



In [ ]:
verExcel_for_osm[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_osm

Geolocalizando con ArcGis

In [8]:
api_arcgis = "AAPKe017698084d742049005e493d4bce9ddY45j51f68jQMWf7cA43OuO3zs160_aYigr1xskjzOzm23Y8n3ijvKA5DlYurkuOq"
geolocator = ArcGIS(auth_domain=api_arcgis)

def tidygeocode(row):
    location = geolocator.geocode(row['noGeo'])
    if location:
        return pd.Series({'latitude':location.latitude,"longitude":location.longitude})
    else:
        return pd.Series({'latitude':None,'longitude':None})
    

In [9]:
verExcel_for_arcgis[['latitude','longitude']] = verExcel_for_osm.apply(tidygeocode,axis=1)

verExcel_for_arcgis

,Linea,Representante,Codigo Medico,Codigo Inst,Tipo Inst,Tipo Pers,Paterno,Materno,Nombre,Calle,...,Da Consulta en Hosp,Fecha_Plan,Hora_Plan,Direccion,Col,Ciudad,Pais,noGeo,latitude,longitude
0,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{331E754A-DF8F-4268-B7A1-725823066B40},{04C32602-3669-4BC0-A11B-9775F0FF8645},CONSULTORIOS,MEDICO,AGUILAR,GARCIA,PAMELA,LUIS LARA PARDO,...,NO,2024-07-16,00:00:00,LUIS LARA PARDO,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico,"LUIS LARA PARDO, JARDIN BALBUENA,VENUSTIANO CA...",19.417284,-99.113956
1,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8A30491F-A3B4-4618-94F3-196F0071BC11},{62247D5F-ABB7-4F0A-8F79-18B6C7ED135A},HOSPITALES,MEDICO,AGUILAR,ZAFRA,RODOLFO,RETORNO 33 1 DE AV. DEL TALLER,...,NO,2024-07-12,11:00:00,RETORNO 33 1 DE AV. DEL TALLER,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico,"RETORNO 33 1 DE AV. DEL TALLER, JARDIN BALBUEN...",19.413122,-99.106587
2,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8E3548FB-1392-48D1-92D9-55508F0D4F4D},{B1614C2D-6B5C-4EA5-993B-7981BFF65C71},CONSULTORIOS,MEDICO,AGUIRRE,EVERARDO,LAURA,SUR 12 422,...,NO,2024-07-04,00:00:00,SUR 12 422,AGRICOLA ORIENTAL,IZTACALCO,Mexico,"SUR 12 422, AGRICOLA ORIENTAL,IZTACALCO,CIUDAD...",19.396208,-99.065600
3,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{62E05F91-6FBE-4217-9CB4-BF0284FFF5B0},{66F41233-AE11-4F6A-9EF1-A4960B903F4D},CONSULTORIOS,MEDICO,AGUIRRE,HERNANDEZ,JOSE RAUL,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,...,NO,2024-07-15,14:00:00,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,GENERAL IGNACIO ZARAGOZA,VENUSTIANO CARRANZA,Mexico,LOCAL 6 MERCADO IGNACIO ZARAGOZA ENTRE CALLE 3...,19.412536,-99.091234
4,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{2D02B950-180B-4C61-8B0C-E4B8010CE36E},{04194637-7EF3-44B0-BCEE-6146126B8DF2},CONSULTORIOS,MEDICO,ALDAPE,MARTINEZ,GABRIELA,J. MARTINEZ ALBA 71,...,NO,2024-07-17,09:00:00,J. MARTINEZ ALBA 71,MOCTEZUMA 1 SECCION,VENUSTIANO CARRANZA,Mexico,"J. MARTINEZ ALBA 71, MOCTEZUMA 1 SECCION,VENUS...",19.431765,-99.109030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{378AB467-DF99-4254-80BC-D2C4D9E2DDC0},{9801E99C-209D-4C6B-900E-41E064D48F87},HOSPITALES,MEDICO,ZAPATA,RIVERA,SAUL,IGNACIO ZARAGOZA 185,...,NO,2024-07-15,17:00:00,IGNACIO ZARAGOZA 185,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico,"IGNACIO ZARAGOZA 185, JARDIN BALBUENA,VENUSTIA...",19.424218,-99.105263
218,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{6A8660F8-D882-4A9A-B6CE-E7209F662353},{466C8793-3EC1-4878-A69C-22B0F0D1C5FB},HOSPITALES,MEDICO,ZAVALA,CASTANEDA,RAUL SERGIO,VICENTE GUERRERO 78,...,NO,2024-06-27,13:11:00,VICENTE GUERRERO 78,APATLACO,IZTACALCO,Mexico,"VICENTE GUERRERO 78, APATLACO,IZTACALCO,CIUDAD...",19.380168,-99.113931
219,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{8414AF75-B338-4D00-ABDA-1F2340CB905A},{8FA76F24-38FB-4F2B-BE3A-7355CD7BC4A6},CONSULTORIOS,MEDICO,ZENDEJAS,MARTINEZ,CLAUDIA MARIANA,ORIENTE 168 # 134,...,NO,2024-07-18,10:00:00,ORIENTE 168 # 134,MOCTEZUMA 1 SECCION,VENUSTIANO CARRANZA,Mexico,"ORIENTE 168 # 134, MOCTEZUMA 1 SECCION,VENUSTI...",19.428589,-99.097454
220,RAYERE,P1304 - MARTINEZ PORTILLO VICTOR MANUEL,{69E77EA6-F44B-477F-B515-5E580AE5B548},{9801E99C-209D-4C6B-900E-41E064D48F87},HOSPITALES,MEDICO,ZENTENO,FUENTES,RODRIGO,IGNACIO ZARAGOZA 185,...,NO,2024-07-11,17:00:00,IGNACIO ZARAGOZA 185,JARDIN BALBUENA,VENUSTIANO CARRANZA,Mexico,"IGNACIO ZARAGOZA 185, JARDIN BALBUENA,VENUSTIA...",19.424218,-99.105263


<p>Se creo un Dataframe con geometria y con una Referencia de sistema de Coordenadas.   en este caso tenemos un sistema de coordenasas ITRF 2008 y Geometria de Puntos.<p>

<a href="https://spatialreference.org/ref/">Spatial Reference</a>

In [15]:
geo_rawData_ITRF = gpd.GeoDataFrame(data=verExcel_for_arcgis,geometry=gpd.points_from_xy(verExcel_for_arcgis.longitude,verExcel_for_arcgis.latitude),crs='EPSG:5332')

geo_rawData = gpd.GeoDataFrame(data=verExcel_for_arcgis,geometry=gpd.points_from_xy(verExcel_for_arcgis.longitude,verExcel_for_arcgis.latitude),crs='EPSG:4326')

Abrir mapa para mostrar datos.

In [ ]:
geo_rawData.explore()

Convertir el archivo a un GeoJson o Un Shapefile

In [ ]:
#geo_rawData.to_file('MediccosP1304.shp')
geo_rawData.to_file('Medicos'+CodRuta+".geojson",driver='GeoJSON')
geo_rawData_ITRF.to_file('Medicos'+ CodRuta+'_MEX.geojson',driver='GeoJSON')
verExcel_for_arcgis.to_excel("Medicos"+CodRuta+"cCoordenadas.xlsx")
